## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('..')
import stober_synthesis_utils as stober

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2024_10_10_random3try2.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [8]:
#jubilee.home_all()
logger.info('Homed Jubilee')

In [5]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [6]:
sample_table = pd.read_csv('SampleTable_2024_11_04_OptimizationRound1.csv')

In [7]:
stober.count_stock_vials(sample_table, 15000)

{'teos_count': 1, 'ammonia_count': 1, 'water_count': 1, 'ethanol_count': 3}

In [8]:
sample_table['ethanol_volume'].sum()/10000

3.23776509296149

## Load Tools

In [9]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [10]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [11]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [61]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [62]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [64]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 10)

In [65]:
water_syringe.set_pulsewidth(water_syringe.full_position+150, s = 500)

In [66]:
water_syringe.load_syringe(750, water_syringe.full_position+150)

Loaded syringe, remaining volume 750 uL


## Load ammonia syringe

In [67]:
ammonia_syringe.set_pulsewidth(1420, s = 10)

In [68]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [69]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [70]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [71]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+350, s = 500)

In [72]:
ammonia_syringe.load_syringe(370, ammonia_syringe.full_position+350)

Loaded syringe, remaining volume 370 uL


## Load TEOS syringe

In [73]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [74]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [75]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 100)

In [76]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+200, s = 200)

In [77]:
teos_syringe.load_syringe(690, teos_syringe.full_position+200)

Loaded syringe, remaining volume 690 uL


## load ethanol syringe

In [71]:
#jubilee.park_tool()

In [78]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [81]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [82]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 10)

In [104]:
mix_syringe.empty_position

1830

In [83]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+400, s = 500)

In [84]:
mix_syringe.load_syringe(3300, mix_syringe.full_position+400)

Loaded syringe, remaining volume 3300 uL


## Load labware


In [23]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(16.5, 171.2), (129.9, 171.2), (130.2, 101.1)])

-0.002139787221987019
New manual offset applied to septavialrev1_44_holder_2000ul


In [24]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(29.2, 260.0),(114.2, 261),(115.2, 205.0)])

-0.014809704144652332
New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [25]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [26]:
teos_stocks = [stocks[0]]
ammonia_stocks = [stocks[1]]
water_stocks = [stocks[2]]
ethanol_stocks = [stocks[3], stocks[4], stocks[5], stocks[6], stocks[7], stocks[8]]
rinse_stocks = [stocks[9], stocks[10], stocks[11]]

In [27]:
samples['A8'].y

171.4426516858021

In [28]:
samples['F8'].y

101.34217023312605

## Check sample alignment

In [66]:
jubilee.pickup_tool(mix_syringe)
for well in stocks:
    jubilee.move_to(x = well.x, y = well.y)
    jubilee.move_to(z = well.top_+7)
    ans = input(f'Is the tip centered over well?')
    if ans == 'y':
        continue
    else:
        continue

Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 



KeyboardInterrupt



In [55]:
jubilee.pickup_tool(ammonia_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    jubilee.move_to(z = loc.top_+2)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

Is the tip centered over sample A1? 
Is the tip centered over sample A2? 
Is the tip centered over sample A3? 
Is the tip centered over sample A4? 
Is the tip centered over sample A5? 
Is the tip centered over sample A6? 
Is the tip centered over sample A7? 
Is the tip centered over sample A8? 
Is the tip centered over sample B1? 
Is the tip centered over sample B3? 
Is the tip centered over sample B4? 
Is the tip centered over sample B5? 
Is the tip centered over sample B6? 
Is the tip centered over sample B8? 
Is the tip centered over sample C1? 
Is the tip centered over sample C2? 
Is the tip centered over sample C3? 
Is the tip centered over sample C4? 
Is the tip centered over sample C5? 
Is the tip centered over sample C6? 


In [ ]:
jubi

## also check all 4 tools move to last position correctly

In [56]:
jubilee.park_tool()

# Experiment


In [85]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}

In [86]:
dilution_table = sample_table.copy()

In [87]:
dilution_table['well'] = dilution_table['dilute_well']

In [88]:
location_lookup_dilute = {row['uuid']:samples[row['well']] for i, row in dilution_table.iterrows()}

## add ethanol

In [81]:
jubilee.park_tool()

In [78]:
mix_syringe.remaining_volume

3874.4048383731597

In [89]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table, location_lookup, 'ethanol_volume', ethanol_stocks, stocks_usable_volume=15000, dwell_time = 10)

In [90]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [91]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [92]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [93]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table, location_lookup, 'ammonia_volume', ammonia_stocks)

## mix precursors

In [94]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup, rinse_stocks, 5)

## Add TEOS and mix

In [95]:
teos_start_time = time.time()

In [96]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table, location_lookup, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks))

### Dilution

In [97]:
hrs5 = 60*60*5
while (time.time() - teos_start_time) < hrs5:
    time.sleep(60)

# dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table, location_lookup_dilute, 'ethanol_dilute_vol', ethanol_stocks[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup[uuid_val]
    dilute_well = location_lookup_dilute[uuid_val]
    vol = row['sample_dilute_vol']
    
    mix_syringe.aspirate(vol, sample_well.bottom(+5))
    time.sleep(10)
    mix_syringe.dispense(vol, dilute_well.bottom(+5))
    time.sleep(10)
    logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')

    for stock in rinse_stocks:
        mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)

    logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

    

In [98]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,f9b8246e-8b80-4955-9560-ada457bad27c,51.404155,27.952415,43.545530,1577.097900,0.531502,8.138741,0.010163,796.0,904.0,A1,C7
1,1,56faa977-e163-444a-9156-e2fa4f7fe338,20.531058,13.736371,12.098204,1653.634367,1.330736,3.250651,0.004097,0.0,1700.0,A2,C8
2,2,da8a1479-69d0-45be-b579-65c7a1dfa21b,19.424944,10.564708,11.428266,1658.582082,1.406513,3.075522,0.003878,0.0,1700.0,A3,D1
3,3,a20c37f0-5b38-4a7c-bee3-d0d3d116c9e0,19.390934,10.795518,14.617081,1655.196466,1.408979,3.070137,0.003870,0.0,1700.0,A4,D2
4,4,c0c257a6-ec89-4274-8d2a-38b479f2eece,52.695785,9.986298,45.473300,1591.844618,0.518475,8.343243,0.010429,819.0,881.0,A5,D3
5,5,0ec4dd81-7487-4689-96b9-b0f646a2047d,19.506982,26.607262,44.223148,1609.662609,1.400597,3.088511,0.003870,0.0,1700.0,A6,D4
6,6,ab43267e-6f46-4c9a-bc6a-ba6240fcb2fb,52.107864,28.135001,45.458031,1574.299103,0.524324,8.250158,0.010298,809.0,891.0,A7,D5
7,7,23709b8a-dfbe-4f00-b9a7-644344ee10e2,25.873608,9.969554,37.543594,1626.613244,1.055957,4.096529,0.005142,0.0,1700.0,A8,D6
8,8,74579cb8-62f3-4d9e-8f66-6ca76a8ff53a,29.221116,26.235873,13.323223,1631.219788,0.934989,4.626535,0.005819,111.0,1589.0,B1,D7
9,9,bfcf29cb-db74-40a1-9d27-911985c3f28f,22.154110,9.392500,9.732500,1658.720890,1.233244,3.507626,0.004423,0.0,1700.0,B3,D8
